In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
import os

In [2]:
# using the same model as in the embedding step
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")

C:\Users\zhong\AppData\Local\Temp\ipykernel_17576\4034726174.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")


In [5]:
# Load FAISS vector store from saved index
index_path = "../vectorstore/faiss_index"

vectorstore = FAISS.load_local(
    folder_path="../vectorstore/faiss_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True  # ✅ only if you trust the file!
)

In [6]:
retriever = vectorstore.as_retriever()

### Run a query

In [164]:
# query = "What's OPT? What's the process for applying one?"

# query = "can I re-enter US after visiting Canada on F1 visa?"

# query = "my ead card starts in february 2025, when are my reports due?"

query = "what is the process for switching from F1 to H1B visa? what should I summit to USCIS and OIE for this?"

In [165]:
load_dotenv()
api_key = os.getenv("TOGETHER_API_KEY")

In [166]:
retrieved_docs = retriever.invoke(query)

In [167]:
print(f"{len(retrieved_docs)} docs retrieved")

4 docs retrieved


In [168]:
retrieved_docs

[Document(id='5570162a-de77-4a04-817b-ce7981603560', metadata={'source': 'html_oie_employment_resources_f1-to-h1b-status.txt'}, page_content='Report Change of Status from F-1 to H-1B\nIf you have changed your status from F-1 to H-1B, you must provide specific documentation to OIE by submitting the "\nChange of Status from F-1 (OPT/STEM OPT)\n" e-Form on the\nMyOIE portal\n.\nIf you were approved for H-1B as a change of status, please provide scanned copies of the following:\nH-1B Approval Notice (I-797A)\nSTEM OPT final Evaluation, found on Page 5 of the I-983 (for students on the 24-month STEM OPT only)\nIf you were approved for H-1B by consular processing, please provide scanned copies of the following:\nH-1B Approval Notice (I-797B)\nEAD card\nMost recent I-94 reflecting your entry as H-1B\nSTEM OPT Final Evaluation, found on Page 5 of the\nI-983\n(for students on the 24-month STEM OPT only)\nThese materials are\ndue within 10 days of your\xa0H-1B status start date and must be submi

In [169]:
combined_docs_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [170]:
prompt_template = """
You are an expert assistant. Use the context below to answer the user's question.
Do NOT include any internal thoughts or explanations.

Context:
{documents}

User Question:
{query}

Answer:
"""

In [171]:
prompt = PromptTemplate(
    input_variables=["documents", "query"],
    template=prompt_template
)

In [172]:
online_llm = ChatOpenAI(
    model="meta-llama/Llama-Vision-Free",
    api_key=api_key,
    base_url="https://api.together.xyz/v1",
    temperature=0
)

In [173]:
online_llm_chain = prompt | online_llm


# This took 2 s to generate summary
online_llm_summary = online_llm_chain.invoke(
    {
        "documents": combined_docs_text,
        "query": query
    }
).content

In [174]:
print(online_llm_summary)

To switch from F-1 to H-1B visa, you must submit the following documents to USCIS and OIE:

**To USCIS:**

* H-1B petition with supporting documents, including:
	+ H-1B Approval Notice (I-797A or I-797B)
	+ EAD card (if applicable)
	+ Most recent I-94 reflecting your entry as H-1B
	+ STEM OPT Final Evaluation, found on Page 5 of the I-983 (for students on the 24-month STEM OPT only)

**To OIE:**

* Submit the "Change of Status from F-1 (OPT/STEM OPT)" e-Form on the MyOIE portal, which requires:
	+ Approval Notice (I-797)
	+ Completed Final Evaluation (page 5) of the I-983 Form

These materials are due within 10 days of your H-1B status start date.
